## 지도학습  

#### 분류 - 다항분류
- 목적변수의 범주가 2개 이상인것 

In [2]:
import pymongo as mg
import pandas as pd 

In [3]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['db_NHIS'] #Database 선택
collection = database['Join Collection NSC2_BND_M20'] # collection 선택
cursor = collection.find({})
list_M20 = list(cursor) # casting
df_BND_M20 = pd.DataFrame(list_M20) 
df_BND_M20[:2]

,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,OPRTN_YN,MDCARE_DD_CNT,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY
0,64e6bbca5b6c745c7e44d61f,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,0,1,1,3,5.0,0.15,10590,3000,7590,2002
1,64e6bbca5b6c745c7e44d620,294364,1921LE,200707.0,T08-T14,W00-W19,2002090466395,63715,20020910,3,...,0,1,1,2,5.0,0.15,16520,4950,11570,2002


### 전처리
- 목표변수 : 'SICK_SYM1'
- 설명변수 : 'OPRTN_YN','TOT_PRSC_DD_CNT','ED_RC_TOT_AMT', 'EDC_SBA', 'EDC_INSUR_BRDN_AMT'

In [4]:
df_BND_M20['SICK_SYM1'].value_counts()

J209     8244
I10      4629
I109     3275
F_       3203
N_       3177
         ... 
H700        1
E058        1
M5397       1
T18         1
P289        1
Name: SICK_SYM1, Length: 3414, dtype: int64

In [5]:
len('F_'), len('J209')

(2, 4)

In [6]:
'J209'[1:], str('I109')[:3]

('209', 'I10')

In [7]:
def convertSICK_SYM(sick_sym) :
    if len(sick_sym) < 3 : 
        return None              ## 크기가 3보다 작은 null
    else : 
        return sick_sym[:3]          ## 3보다 크면 3자리에서 자름
    
## 3개를 자르는 이유는 대분류3자리 소분류 2자리로 나눠져있음 

In [8]:
convertSICK_SYM('J209'), convertSICK_SYM('F_')

('J20', None)

In [9]:
 df_BND_M20['SICK_SYM1_3'] = df_BND_M20['SICK_SYM1'].apply(convertSICK_SYM)

In [10]:
 df_BND_M20['SICK_SYM1_3'].value_counts()

J20    8917
I10    8427
J03    4187
M54    3904
J06    3460
       ... 
G97       1
K01       1
S28       1
I72       1
P28       1
Name: SICK_SYM1_3, Length: 747, dtype: int64

In [11]:
df_BND_M20_dropna = df_BND_M20.dropna()
df_BND_M20_dropna.isnull().sum()

_id                   0
RN_INDI               0
BTH_YYYY              0
DTH_YYYYMM            0
COD1                  0
COD2                  0
RN_KEY                0
RN_INST               0
MDCARE_STRT_DT        0
FORM_CD               0
MCARE_SUBJ_CD         0
SICK_SYM1             0
OFIJ_TYPE             0
OPRTN_YN              0
MDCARE_DD_CNT         0
VSHSP_DD_CNT          0
TOT_PRSC_DD_CNT       0
MCARE_RSLT_TYPE       0
EDC_ADD_RT            0
ED_RC_TOT_AMT         0
EDC_SBA               0
EDC_INSUR_BRDN_AMT    0
STD_YYYY              0
SICK_SYM1_3           0
dtype: int64

In [12]:
target_list = df_BND_M20_dropna['SICK_SYM1_3'].value_counts().index.to_list() ## 값에 따른 인텍스만 추출

In [13]:
target_list.index('L02'), target_list.index('J03')

(83, 6)

In [14]:
def applyIndexNumber(sick_sym_3) :
    inexNumber = target_list.index(sick_sym_3)
    return inexNumber

In [15]:
df_BND_M20_dropna['SICK_SYM1_3_target'] = df_BND_M20_dropna['SICK_SYM1_3'].apply(applyIndexNumber)
df_BND_M20_dropna[:2]

C:\Users\02-19\AppData\Local\Temp\ipykernel_5960\836018491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BND_M20_dropna['SICK_SYM1_3_target'] = df_BND_M20_dropna['SICK_SYM1_3'].apply(applyIndexNumber)


,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY,SICK_SYM1_3,SICK_SYM1_3_target
0,64e6bbca5b6c745c7e44d61f,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,1,3,5.0,0.15,10590,3000,7590,2002,L02,83
3,64e6bbca5b6c745c7e44d622,294364,1921LE,200707.0,T08-T14,W00-W19,2002020488631,23506,20020208,3,...,1,3,5.0,0.15,12920,3000,9920,2002,K29,8


### 정형화

In [16]:
df_BND_M20_dropna.columns

Index(['_id', 'RN_INDI', 'BTH_YYYY', 'DTH_YYYYMM', 'COD1', 'COD2', 'RN_KEY',
       'RN_INST', 'MDCARE_STRT_DT', 'FORM_CD', 'MCARE_SUBJ_CD', 'SICK_SYM1',
       'OFIJ_TYPE', 'OPRTN_YN', 'MDCARE_DD_CNT', 'VSHSP_DD_CNT',
       'TOT_PRSC_DD_CNT', 'MCARE_RSLT_TYPE', 'EDC_ADD_RT', 'ED_RC_TOT_AMT',
       'EDC_SBA', 'EDC_INSUR_BRDN_AMT', 'STD_YYYY', 'SICK_SYM1_3',
       'SICK_SYM1_3_target'],
      dtype='object')

In [17]:
target =df_BND_M20_dropna['SICK_SYM1_3_target']
features = df_BND_M20_dropna[['EDC_ADD_RT', 'ED_RC_TOT_AMT']]
target.shape, features.shape

((829,), (829, 2))

#### split

In [18]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=111)
features_train.shape, target_train.shape, features_test.shape, target_test.shape

((621, 2), (621,), (208, 2), (208,))

### 모델 학습
- 목표변수가 범주형일 경우 : DecisionTreeClassifier
- 목표변수가 연속형일 경우 : DecisionTreeRegressor

In [19]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

In [20]:
model = DecisionTreeClassifier() # 인스턴스화

In [21]:
from sklearn.model_selection import GridSearchCV

In [39]:
hyper_params = {'min_samples_leaf' : [5, 7, 9]
               ,'max_depth' : [9, 11]
               ,'min_samples_split' : [5, 6, 7]}
# hyper_params = {'min_samples_leaf' : range(2,5) #[2,3,4,5]
#                ,'max_depth' :  range(2,5)
#                ,'min_samples_split' :  range(2,5)}

### 평가 score 분류 
- Default 값
    - 분류 : 정확도(Accuracy)
    - 예측 : R squre(R-squre)

In [40]:
from sklearn.metrics import f1_score, make_scorer

In [41]:
scoring = make_scorer(f1_score)

In [42]:
grid_search = GridSearchCV(model, param_grid=hyper_params, cv=3, verbose=1, scoring=scoring) 
#verbos=1 학습되는 내용을 출력하겠다.
# scoring=scoring -->  defalt 값은 accay score / 원하는 값으로 설정 --> 현재는 f1_score으로 최상의 값을  찾음

In [43]:
grid_search.fit(features_train, target_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 234, in __call__
    return self._score(
           ^^^^^^^^^^^^
  File "C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
                        ^^^^^^^^^^^^

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [9, 11], 'min_samples_leaf': [5, 7, 9],
                         'min_samples_split': [5, 6, 7]},
             scoring=make_scorer(f1_score), verbose=1)

In [44]:
grid_search.best_estimator_

DecisionTreeClassifier(max_depth=9, min_samples_leaf=5, min_samples_split=5)

In [45]:
grid_search.best_score_, grid_search.best_params_ 
# best_score_ : 0.148148 데이터 정확도가 떨어진다는 것을 확인

(nan, {'max_depth': 9, 'min_samples_leaf': 5, 'min_samples_split': 5})

In [30]:
best_model = grid_search.best_estimator_ # 하나의 모델 --> 그 중에서 최고의 모델
best_model  

DecisionTreeClassifier(max_depth=9, min_samples_leaf=5, min_samples_split=5)

In [31]:
target_test_predict = best_model.predict(features_test)
target_test_predict

array([18,  2,  0,  1,  2,  2, 13,  0,  2,  3,  0,  9,  0,  1,  9, 34,  0,
        8, 55,  0,  4, 14,  6,  7,  9,  2, 34,  9,  2,  6,  2,  0,  5,  0,
        0,  1,  9,  3,  9,  0, 10,  1, 42, 10, 13,  0, 20,  1,  1,  0,  1,
        7, 11,  0, 34,  0, 34, 21, 10,  1,  1, 55,  0,  9,  0, 12, 55,  3,
       12,  1, 11,  1,  4, 10, 10, 20,  2,  1,  1,  6, 55,  0, 11,  1, 12,
        5, 20,  0,  3,  1,  0,  1,  0,  1, 11,  0,  9,  0,  1,  1,  6,  0,
       21,  7, 25,  6,  3,  4, 15,  4,  7, 12,  5,  0,  0, 21, 15,  2, 34,
        7,  0,  4, 55,  9,  2,  0,  0,  2, 20,  5,  5,  2,  0, 11,  0,  0,
       25,  0, 18,  6,  6,  4,  2,  0,  2,  1,  0,  6,  0,  9,  6, 14,  1,
        2, 30, 12,  5,  1,  2, 10,  4, 18,  2,  0,  0,  4, 14,  6,  3,  2,
        5, 30,  1,  0,  0,  0,  0,  2, 18,  4,  0, 14, 20,  1,  2,  5, 18,
        0,  4,  3,  2,  1,  1,  0,  4,  1,  1, 12,  1,  0,  5,  5,  6,  0,
        4, 34,  1, 10], dtype=int64)

### 평가

In [32]:
from sklearn.metrics import classification_report

In [33]:
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.09      0.27      0.13        15
           1       0.07      0.29      0.11         7
           2       0.43      0.45      0.44        20
           3       0.00      0.00      0.00         6
           4       0.42      0.62      0.50         8
           5       0.10      0.25      0.14         4
           6       0.00      0.00      0.00         4
           7       0.60      0.50      0.55         6
           8       1.00      0.10      0.18        10
           9       0.10      0.20      0.13         5
          10       0.14      0.20      0.17         5
          11       0.00      0.00      0.00         2
          12       0.67      1.00      0.80         4
          13       0.50      0.50      0.50         2
          14       0.50      0.50      0.50         4
          15       0.00      0.00      0.00         3
          16       0.00      0.00      0.00         2
          17       0.00    

C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\metrics\_cla